RAG 용 DB만드는 코드

In [ ]:
import os
from dotenv import load_dotenv
import openai
from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage, SystemMessage
import time
import re
import PyPDF2


temp = 0
for filename in os.listdir(directory_path): # 법 문서 들어있는 파일 위치
    print(filename)

    
    pdf_path = '/home/dilab05/work_directory/김재성/진짜마지막법률/' + filename    
    try:
        text = extract_text_from_pdf(pdf_path)
        pattern = re.compile(r'제1조\(목적\)(.*?)제2', re.DOTALL)
        match = pattern.search(text)


        if match:
            extracted_text = match.group(1).strip()
            cleaned_text = extracted_text.replace('\n', '')
        else:
            print(filename)


    
        doc_text = cleaned_text
        doc_metadata = {"file_name": filename}
        document_1 = Document(text=doc_text, metadata=doc_metadata)
        dse.append(document_1)
        qqqqqq.append(filename)
    except:
        print("sssssss")


Settings.embed_model = resolve_embed_model("local:BAAI/bge-m3")
embed_model = resolve_embed_model("local:BAAI/bge-m3")

index = VectorStoreIndex.from_documents(
dse, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=chat,similarity_top_k=1)
retriever = index.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=1,
    vector_store_kwargs={"mmr_threshold": 0.5},
)



In [56]:
chat = ChatUpstage(api_key="내 api key") # upstage api key

'5ㆍ18민주화운동 등에 관한 특별법(법률)(제18465호)(20220701).pdf'

In [60]:

def rag_for_law(text):
    
    question_1 = text
    template = f"""

질문: {question_1}

질문과 관련된 법률을 찾아줘
"""

    nodes = retriever.retrieve(
        question_1
    )
    temp = []
    for n in nodes:
        temp.append(n.metadata['file_name'])
        
    return temp
    
def extract_between_asterisks(text):
    pattern = r'"(.*?)"'
    matches = re.findall(pattern, text)
    return matches

def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        full_text = "".join(page.extract_text() for page in reader.pages) + "\n"
    return full_text

def find_law_names(question_text):
    question = [
        SystemMessage(content='''다음의 질문을 잘 읽고 질문을 해결할 수 있는 법 명을 말하세요. 또한 법률명은 따옴표 안에 표시하세요. 당신의 지식을 믿지 말고 그럴듯한 법률명을 말하세요.'''),
        HumanMessage(content=f'''다음의 질문을 잘 읽고 질문을 해결할 수 있는 법 명을 말하세요. 또한 법률명은 "따옴표" 안에 표시하세요. 질문: {question_text}.''')
    ]
    response = chat.invoke(question, temperature=0)
    content_match = re.search(r"content='(.*?)'", str(response))
    if content_match:
        response = content_match.group(1)
    return extract_between_asterisks(response)

def process_selected_files(selected_files, question_text, directory_path):
    final_responses = []
    for pdf in selected_files:
        pdf_path = os.path.join(directory_path, pdf.strip())
        text = extract_text_from_pdf(pdf_path)[:15000]  
        question = [
            SystemMessage(content="You are a legal expert. You need to read the questions and legal material carefully and provide legal answers based on the contents of the legal material. Additionally, you should only provide answers related to the legal material. Additionally, the legal provisions must be specified.\n만약 questions과 관계없는 legal material이 들어온다면 답변을 하지 않아도 됩니다. \n답변은 다음의 형식을 따라야 합니다: (1) 사실관계 - (2) 관련 법률의 일반 내용 – (3) 사실관계의 적용 – (4) 결론(요약). 답변은 반드시 legal material에 있는 내용을 참고하여 합니다."),
            HumanMessage(content=f"legal material: {text}\n\nquestions: {question_text}\n\n답변은 다음의 형식을 따라야 합니다: (1) 사실관계 - (2) 관련 법률의 일반 내용 – (3) 사실관계의 적용 – (4) 결론.\n(1)을 제외한 (2), (3), (4)는 legal material의 내용을 기반해서 답변합니다. 답변은 반드시 legal material에 있는 내용을 기반해서 해야 합니다.")
        ]
        response = chat.invoke(question, temperature=0)
        content_match = re.search(r"content='(.*?)'", str(response))
        if content_match:
            response = content_match.group(1)
        final_responses.append(response)
    return final_responses

def compile_expert_opinions(final_responses):
    compiled_opinions = '\n\n\n'.join(f"Expert opinion: {response}" for response in final_responses)
    question = [
        SystemMessage(content="You are a legal expert. Expert opinions are provided, you have to compile the opinions of experts, focusing on legal aspects. Please answer in Korean. Lastly, you must mention all the names of the laws you referred to. \n답변은 다음의 형식을 따라야 합니다: (1) 사실관계(법 적용과 관련 된 내용을 담은 사건개요 간략히 설명) - (2) 관련 법률의 일반 내용(- 제1조 - ① (제1항) - 1. (제1호) - 가. (가목)) – (3) 사실관계의 적용 – (4) 결론(요약)"),
        HumanMessage(content=f"전문가 답변: {compiled_opinions}.\n\n전문가들의 답변을 통합하세요. 답변은 다음의 형식을 따라야 합니다: (1) 사실관계 - (2) 관련 법률의 일반 내용 – (3) 사실관계의 적용 – (4) 결론")
    ]
    response = chat.invoke(question, temperature=0)
    content_match = re.search(r"content='(.*?)'", str(response))
    if content_match:
        response = content_match.group(1)
    return response

def main():
    directory_path = '/home/dilab05/work_directory/김재성/진짜마지막법률' # 법률 문서들 있는 위치
    
    for k in range(1):
        rrr = "누가 지하철에서 나를 성추행했는디 법적으로 어떤 처벌을 받지?" # 여따 질문 넣기 input에 *법적 처벌이나 '법적'을 명시해서 넣어주기* EX)누가 지하철에서 나를 성추행했는디 법적으로 어떤 처벌을 받지?
        question_text = rrr

        result = find_law_names(question_text)
        temp = rag_for_law(question_text)
        result += temp
        print(temp)
        selected_files = ["형법(법률)(제19582호)(20240209).pdf", "민법(법률)(제19409호)(20240517).pdf"]
        file_checks = {"민법": 0, "형법": 0, "상법": 0}
        for law in result:
            for filename in os.listdir(directory_path):
                if law in filename and filename.endswith('.pdf'):
                    if law in file_checks:
                        if file_checks[law] == 0:
                            selected_files.append(filename)
                            file_checks[law] = 1
                    else:
                        selected_files.append(filename)
        print('질문: '+ rrr)
        print("\n\n")
        print("찾은 법률문서: " + str(selected_files)) 
        print("\n\n")
        final_responses = process_selected_files(selected_files, question_text, directory_path)
        
        compiled_opinion = compile_expert_opinions(final_responses)

        print(compiled_opinion) # 결과 

if __name__ == "__main__":
    main()

['검사의 국선변호사 선정 등에 관한 규칙(법무부령)(제01075호)(20240403).pdf']
질문: 누가 지하철에서 나를 성추행했는디 법적으로 어떤 처벌을 받지?



찾은 법률문서: ['형법(법률)(제19582호)(20240209).pdf', '민법(법률)(제19409호)(20240517).pdf', '성폭력범죄의 처벌 등에 관한 특례법 시행령(대통령령)(제34158호)(20240125).pdf', '성폭력범죄의 처벌 등에 관한 특례법 시행규칙(법무부령)(제00204호)(20220705).pdf', '성폭력범죄의 처벌 등에 관한 특례법(법률)(제19743호)(20240125).pdf', '검사의 국선변호사 선정 등에 관한 규칙(법무부령)(제01075호)(20240403).pdf']



(1) 사실관계: 지하철에서 모르는 사람이 나를 성추행했습니다.\n\n(2) 관련 법률의 일반 내용:\n\n* 성폭력범죄의 처벌 등에 관한 특례법(성폭력처벌법) 제11조(공중밀집장소에서의 추행)는 버스, 지하철 등 공중이 밀집하는 장소에서 사람을 추행한 사람은 1년 이하의 징역 또는 300만원 이하의 벌금에 처한다고 규정하고 있습니다.\n* 형법 제298조(강제추행)는 폭행 또는 협박으로 사람에 대하여 추행을 한 자는 10년 이하의 징역 또는 1천500만원 이하의 벌금에 처한다고 규정하고 있습니다.\n\n(3) 사실관계의 적용:\n\n* 지하철은 공중이 밀집하는 장소에 해당하며, 모르는 사람이 성추행을 한 행위는 공중밀집장소에서의 추행죄에 해당합니다.\n* 성추행의 정도가 심각하거나, 피해자가 13세 미만인 경우, 형법 제298조에 따라 강제추행죄로 처벌될 수 있습니다.\n\n(4) 결론:\n\n* 지하철에서 모르는 사람이 성추행을 한 행위는 성폭력처벌법 제11조에 따라 공중밀집장소에서의 추행죄로 처벌될 수 있습니다.\n* 성추행의 정도가 심각하거나, 피해자가 13세 미만인 경우, 형법 제298조에 따라 강제추행죄로 처벌될 수 있습니다.


법률 언어 모델 코드
